### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 방대한 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 사전학습(pre‑training)된 '
 '트랜스포머(Transformer) 모델입니다. 사전학습 후에는 인간 피드백을 기반으로 보상 모델을 만들고, 이를 통해 정책을 최적화하는 '
 '강화학습(RLHF) 과정을 거쳐 대화에 적합하도록 미세조정됩니다. 이렇게 학습된 파라미터들은 입력 문맥을 고려해 가장 자연스럽고 의미 '
 '있는 답변을 생성하도록 작동합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [3]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 데이터셋을 이용해 **자기 지도 학습**(self‑supervised learning) 방식으로 사전 '
 '학습(pre‑training)한다.  \n'
 '2. 사전 학습 단계에서 모델은 문맥을 예측하도록 토큰을 마스킹하거나 다음 토큰을 맞추는 과제를 수행해 언어 패턴과 의미 관계를 '
 '학습한다.  \n'
 '3. 이후 특정 작업에 맞게 **지도 학습**(fine‑tuning)이나 강화 학습(RLHF) 등을 통해 사용자 의도에 부합하도록 미세 '
 '조정한다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 장점 요약  \n'
 '\n'
 '- **다양한 주제와 언어에 대한 높은 이해도**: 방대한 텍스트와 다양한 도메인 데이터를 학습해 폭넓은 지식과 문맥 파악 능력을 '
 '갖춤.  \n'
 '- **자연스러운 대화 흐름**: 문맥을 유지하면서 일관된 답변을 생성하고, 질문 의도를 파악해 적절한 수준과 톤으로 응답함.  \n'
 '- **확장성과 커스터마이징**: 사전 학습된 모델을 기반으로 특정 분야에 맞게 파인튜닝하거나, 플러그인·API를 통해 다양한 '
 '애플리케이션에 쉽게 통합 가능.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT와 비슷한 AI 모델 (영어 모델명)\n'
 '\n'
 '- **GPT‑4** (OpenAI)  \n'
 '- **Claud

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Anima", "count": 4},
    {"model_name": "Figma", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Anima 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'Figma 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT‑4는 방대한 인터넷 텍스트 데이터를 수집해, 다음에 올 단어를 예측하도록 스스로 학습하는 **자기지도(pre‑training)** '
 '과정을 거칩니다. 이후 인간이 만든 질문‑답변 쌍이나 피드백을 이용해 모델의 출력을 조정하는 **지도학습(fine‑tuning)** 및 '
 '인간 피드백을 반영해 보상을 최적화하는 **강화학습(RLHF)** 을 적용해 실제 사용 상황에 맞는 응답 품질을 높입니다. 이렇게 두 '
 '단계의 학습을 반복하면서 모델은 언어 구조와 의미를 일반화하고, 다양한 과업에 적용 가능한 범용 인공지능으로 발전합니다.')
<class 'str'> Anima 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Anima 모델은 대규모 텍스트 데이터를 사전 학습(pre‑training)하여 언어의 통계적 패턴과 의미 관계를 학습합니다.  \n'
 '그 후, 특정 작업에 맞게 라벨이 부착된 데이터셋을 이용해 지도 학습(supervised fine‑tuning)이나 인간 피드백을 반영한 '
 '강화 학습(RLHF)으로 성능을 최적화합니다.  \n'
 '학습 과정에서는 손실 함수(loss function)를 최소화하도록 가중치를 역전파(backpropagation)하고, 옵티마이저가 '
 '파라미터를 업데이트합니다.  \n'
 '이러한 단계들을 반복하면서 모델은 점차 더 정확하고 일관된 언어 이해·생성 능력을 갖추게 됩니다.')
<class 'str'> Figma 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Figma 모델은 대량의 이미지와 텍스트 데이터 쌍을 이용해 사전 학습됩니다.  \n'
 '학습 과

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [6]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델의 학습 원리  
(한국어로 자세히 설명합니다)

---

### 1. 전체 흐름

ChatGPT와 같은 대형 언어 모델(LLM)은 크게 **세 단계**로 학습됩니다.

| 단계 | 목적 | 주요 방법 |
|------|------|-----------|
| **① 사전학습 (Pre‑training)** | 방대한 텍스트 데이터를 통해 **언어의 일반적인 패턴**을 습득 | 자기지도 학습 (self‑supervised), 토큰 예측 (next‑token prediction) |
| **② 지도학습 (Supervised Fine‑tuning, SFT)** | 모델을 **대화형** 혹은 **특정 태스크**에 맞게 조정 | 인간이 만든 질문‑답변 쌍을 이용해 지도학습 |
| **③ 인간 피드백 기반 강화학습 (RLHF)** | 모델이 **사용자 의도**에 맞는, **친절하고 안전한** 출력을 생성하도록 최적화 | 인간 평가자·시스템이 만든 보상 모델을 이용해 강화학습 수행 |

각 단계는 앞 단계에서 얻은 지식을 바탕으로 점진적으로 특화됩니다.

---

### 2. 사전학습 (Pre‑training)

#### 2.1 데이터
- **규모**: 수조(token) 수준, 웹 페이지, 책, 논문, 위키피디아, 포럼, 코드 저장소 등 다양한 출처
- **전처리**:  
  - **토크

In [7]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델이 어떻게 학습되는지 단계별로 살펴보기  

아래 내용은 **Transformer 기반 대규모 언어 모델**(예: GPT‑4, GPT‑3.5 등)이 훈련되는 전형적인 과정을 한국어로 정리한 것입니다. 핵심 흐름은 크게 **데이터 준비 → 사전 학습(Pre‑training) → 지도 학습( supervised fine‑tuning) → 강화 학습(RLHF)** 로 나눌 수 있습니다.

---

### 1. 데이터 준비 (Corpus Construction)

| 요소 | 설명 |
|------|------|
| **대규모 텍스트 코퍼스** | 인터넷 웹페이지, 책, 논문, 뉴스, 위키피디아, 포럼, 코드 저장소 등 다양한 출처에서 수십억 토큰 규모의 텍스트를 수집합니다. |
| **클렌징 & 필터링** | 저작권 위반, 개인 정보, 폭력·혐오 표현 등 부적절한 내용은 자동·수동으로 제거합니다. 또한 중복 문서, HTML 태그, 스크립트 등 비텍스트 요소를 정리합니다. |
| **토크나이징 (Tokenization)** | 텍스트를 **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece**와 같은 서브워드 토크나이저로 변환합니다. 하나의 토큰은 보통 1~4개의 문자(영문 기준) 또는 한글 자모·음절을 포함합니다. |
| **데이터 샤플링** | 학습 효율과 편향 감소를 위해 전체 코퍼스를 여러 작은 청크(chunk)로 나누고, 각 청크를 무작위로 섞어 학습 배치를 구성합니다. |

---

### 2. 사전 학습 (Pre‑training) – **자기지도 학습 (Self‑Supervised Learning)**  

#### 2.1 목표 함수 (Loss Function)
- **다음 토큰 예측 (Causal Language Modeling)**  
  \[
  \mathcal{L}_{\text{LM}} = -\sum_{t=1}^{T}\log P\big(x_t \mid x_{<t}; \t

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [8]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

## Deep Learning – A Clear, Detailed Overview

### 1. What It Is, in One Sentence  
**Deep learning** is a sub‑field of machine learning that uses **artificial neural networks with many (deep) layers** to automatically learn hierarchical representations of data, enabling computers to perform tasks such as image recognition, speech transcription, language translation, and more—often with performance comparable to or surpassing human experts.

---

### 2. Why “Deep”?

| Concept | Explanation |
|---------|-------------|
| **Neural network** | A computational model loosely inspired by the brain’s network of neurons. Each “neuron” receives inputs, applies a weighted sum, passes the result through a non‑linear **activation function**, and outputs a value. |
| **Depth** | The number of *hidden* layers (layers between input and output). A **deep** network has many hidden layers (typically ≥ 3, often dozens or hundreds). Depth lets the model build **progressively more abstract features**: low‑l

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [9]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑합니다.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [10]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
**태양계 행성 간략 정리**

| 순서(태양으로부터) | 행성 | 종류 | 평균 지름 (km) | 특징 |
|-------------------|------|------|----------------|------|
| 1 | **수성 (Mercury)** | 암석형 행성 | 4,880 | 가장 작은 행성, 대기가 거의 없으며 표면 온도 차이가 극심함 |
| 2 | **금성 (Venus)** | 암석형 행성 | 12,104 | 두꺼운 이산화탄소 대기와 황산 구름, 온실 효과로 표면 온도 ~470 °C |
| 3 | **지구 (Earth)** | 암석형 행성 | 12,742 | 물이 액체 상태로 존재하는 유일한 행성, 풍부한 대기와 생명 |
| 4 | **화성 (Mars)** | 암석형 행성 | 6,779 | 얇은 이산화탄소 대기, 적색 토양, 과거에 물이 흐른 흔적 |
| 5 | **목성 (Jupiter)** | 가스 행성 | 139,822 | 태양계 최대의 행성, 거대한 폭풍(대적점)과 수많은 위성 |
| 6 | **토성 (Saturn)** | 가스 행성 | 116,464 | 눈에 띄는 고리 시스템, 80여 개 이상의 위성 |
| 7 | **천왕성 (Uranus)** | 얼음 거대 행성 | 50,724 | 자전축이 약 98° 기울어져 있어 ‘옆으로 누운’ 행성, 푸른색은 메탄 가스 |
| 8 | **해왕성 (Neptune)** | 얼음 거대 행성 | 49,244 | 가장 강한 바람(시속 2,400 km)과 눈에 띄는 청색, 트위터(Triton) 위성 보유 |

**주요 포인트**

- **암석형 행성**(수성, 금성, 지구, 화성) : 고체 표면과 얇은 대기(또는 없음).  
- **가스 행성**(목성, 토성) : 주로 수소·헬륨으로 이루어져 있으며, 핵은 존재하지만 표면이 명확하지 않음.  
- **얼음 거대 행성**(천왕성, 해왕성) : 가스 행성보다 물, 암모니아, 메탄

##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [13]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "소프트웨어융합학과를 재학중인데 졸업 후 요즘 AI때문에 미래를 생각했을때 취업하기 좋은 직업을 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [14]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: **가을에 주로 나타나는 대표적인 지구과학 현상**  

| # | 현상 | 왜 가을에 많이 나타나는가? (주요 메커니즘) | 주요 영향·관측 특징 |
|---|------|---------------------------------------------|-------------------|
| 1 | **태풍(열대 저기압) 발생·접근** | • 7·8월에 가장 활발한 열대 저기압이 형성되지만, **가을 초(9월~10월)** 은 남서쪽 태평양의 해수면 온도가 아직 26 °C ~ 28 °C 로 높아 태풍이 계속 발달할 수 있는 환경이 유지됩니다.<br>• 가을이 되면 서쪽 태평양 고기압(태평양 고기압·동아시아 고기압)이 서서히 남하하면서, 태풍이 북쪽으로 이동·진로가 바뀌어 한반도·동북아 지역에 접근하기 쉬워집니다. | • 강풍·폭우·해일(Storm surge) 등 재해 위험이 급증.<br>• 기상청·위성·레이다를 통한 실시간 추적이 활발히 이루어짐. |
| 2 | **중위도 저기압·전선 활동 강화** (냉난류 교차) | • 가을은 **여름철 고온·습한 대류와 겨울철 차가운 대륙 고기압** 사이의 전이기.<br>• 남쪽에서 따뜻하고 습한 해양성 공기가, 북쪽에서 차가운 대륙성 공기가 만나면서 **전선(특히 한랭 전선)과 저기압**이 자주 형성됩니다.<br>• 제트기류가 남쪽으로 내려오면서 전선의 이동 속도가 빨라지고, 저기압의 강도가 강화됩니다. | • 급격한 기온 강하, 강수(소나기·폭우)와 바람이 동반되는 날씨.<br>• 교통·농업·에너지(난방) 수요에 큰 영향을 미침. |
| 3 | **대기 온도역전·안개·연무 발생** | • 가을 밤에는 **지표면이 급격히 냉각**되고, 위층에 남아 있던 여름의 따뜻한 공기가 위에 머무르게 됩니다.<br>• 이때 **온도역전층**이 형성돼 수직 혼합이 억제되고, 습도가 높은 저층 

In [15]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 가을

 가을에 발생하는 자연 현상:
가을에 주로 발생하는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

*   **편서풍의 계절 변화**: 가을이 되면 북반구에서 낮과 밤의 길이(일교차)가 짧아지게 되며, 편서풍의 위치가 남쪽으로 이동하게 됩니다. 이로 인해 우리나라에는 따뜻한 북태평양 기단과 차가운 오호츠크 해 기단이 만나서 형성된 전선의 영향으로 비가 자주 내리게 됩니다.

*   **성층권 오존층 감소**: 가을이 되면 지구의 자전축에 따른 태양 빛의 입사각도 변화로 인해 오존층이 약화되어 오존층의 구멍이 나타날 수 있습니다. 이로 인해 자외선 차단 효과가 약화되어 인체에 미치는 피해가 증가할 수 있습니다.

*   **북반구 단풍**: 가을이 되면 북반구의 낮의 길이가 짧아지게 되며, 밤과 낮의 일교차가 커지게 됩니다. 이로 인해 나무의 잎이 변색되어 단풍이 발생하게 됩니다. 이러한 단풍은 대개 10월 중순에 절정에 이르게 됩니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [16]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## Summary

Based on the current exchange rate of 1 USD = 1,377.98 KRW, this response provides an analysis of the currency market.

## Analysis

The current exchange rate of 1 USD = 1,377.98 KRW indicates that the Korean won (KRW) is relatively weak compared to the US dollar (USD). A higher exchange rate means that it takes more won to buy one dollar.

## Possible Reasons for the Exchange Rate

Several factors contribute to the current exchange rate:

*   **Economic indicators**: Korea's economic growth rate, inflation rate, and interest rates influence the value of the won.
*   **Global market trends**: The US dollar's value against other major currencies, such as the euro and yen, also affects the KRW/USD exchange rate.
*   **Trade balance**: Korea's trade balance, including exports and imports, impacts the demand for and supply of won and dollars.
*   **Capital flows**: Foreign investment in Korea and Korean inv